# Lab 10.3.1 Visdom Example

**Jonathan Choi 2021**

**[Deep Learning By Torch] End to End study scripts of Deep Learning by implementing code practice with Pytorch.**

If you have an any issue, please PR below.

[[Deep Learning By Torch] - Github @JonyChoi](https://github.com/jonychoi/Deep-Learning-By-Torch)
Here, we are going to learn about the usage of the Visdom, which is the tool to visualize the graphs and other various things during experiment.

You can find more about here:

https://pypi.org/project/visdom/

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

## Import Visdom

Jupyter Notebook > Terminal

Type ```python -m visdom.server```

In [2]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


## Text

In [3]:
vis.text("Hello, world", env="main")

'window_39d440e5bf1724'

### Take a Moment!

**TORCH.RANDN**

torch.randn(*size, *, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False) → Tensor

Returns a tensor filled with random numbers from a normal distribution with mean 0 and variance 1 (also called the standard normal distribution).

out(i)∼N(0,1)
 
The shape of the tensor is defined by the variable argument ```size```.

**Parameters**
> **size** (int...) – a sequence of integers defining the shape of the output tensor. Can be a variable number of arguments or a collection like a list or tuple.

**Keyword Arguments**
> - **generator** (torch.Generator, optional) – a pseudorandom number generator for sampling

> - **out** (Tensor, optional) – the output tensor.

> - **dtype** (torch.dtype, optional) – the desired data type of returned tensor. Default: if None, uses a global default (see torch.set_default_tensor_type()).

> - **layout** (torch.layout, optional) – the desired layout of returned Tensor. Default: torch.strided.

> - **device** (torch.device, optional) – the desired device of returned tensor. Default: if None, uses the current device for the default tensor type (see torch.set_default_tensor_type()). device will be the CPU for CPU tensor types and the current CUDA device for CUDA tensor types.

> - **requires_grad** (bool, optional) – If autograd should record operations on the returned tensor. Default: False.

Example:

```
>>> torch.randn(4)
tensor([-2.1436,  0.9966,  2.3426, -0.6366])
>>> torch.randn(2, 3)
tensor([[ 1.5954,  2.8929, -1.0923],
        [ 1.1719, -0.4709, -0.1996]])
```

## Image

In [4]:
a = torch.randn(3, 200, 200)
vis.image(a)

'window_39d440e5cab6b0'

### Take a Moment!

## Q What is the difference between torch.tensor and torch.Tensor?

Since version 0.4.0, it is possible to use torch.tensor and torch.Tensor

What is the difference? What was the reasoning for providing these two very similar and confusing alternatives?

## A

In PyTorch torch.Tensor is the main tensor class. So all tensors are just instances of torch.Tensor.

When you call torch.Tensor() you will get an empty tensor without any data.

In contrast torch.tensor is a function which returns a tensor. In the documentation it says:

torch.tensor(data, dtype=None, device=None, requires_grad=False) → Tensor
Constructs a tensor with data.

This also also explains why it is no problem creating an empty tensor instance of `torch.Tensor` without `data` by calling:
tensor_without_data = torch.Tensor()
But on the other side:

```tensor_without_data = torch.tensor()```

Will lead to an error:

```---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-12-ebc3ceaa76d2> in <module>()
----> 1 torch.tensor()

TypeError: tensor() missing 1 required positional arguments: "data"
```

But in general there is no reason to choose `torch.Tensor` over `torch.tensor`. Also `torch.Tensor` lacks a docstring.
Similar behaviour for creating a tensor without data like with: torch.Tensor() can be achieved using:

```torch.tensor(())```

Output:

```tensor([])```

## Images

In [5]:
vis.images(torch.Tensor(3,3,28,28))

'window_39d440e5d0187a'

## Example (using MNIST and CIFAR10)

In [6]:
MNIST = datasets.MNIST(root = 'MNIST_data/', 
                       download = True, 
                       transform = transforms.ToTensor(),
                       train = True)
cifar10 = datasets.CIFAR10(root = 'cifar10/', 
                           download = True, 
                           transform = transforms.ToTensor(),
                           train = True)

C:\Users\buddhalight\envs\buddhalight\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Files already downloaded and verified


### CIFAR10

In [7]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.image(data[0], env = 'main')

torch.Size([3, 32, 32])


'window_39d440e64faa3c'

### MNIST

In [8]:
data = MNIST.__getitem__(0)
print(data[0].shape) #make this since data is tuple of (X, Y)
vis.image(data[0], env = "main")

torch.Size([1, 28, 28])


'window_39d440e658134c'

### Check dataset

In [9]:
data_loader = torch.utils.data.DataLoader(dataset = MNIST,
                                          shuffle = False,
                                          drop_last = True,
                                          batch_size = 32)

In [10]:
for num, value in enumerate(data_loader):
    #print(value) tuple of (datas, labels)
    value = value[0] # get only datas
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [11]:
vis.close(env="main")

''

In [19]:
Y_data = torch.randn(5)
plt = vis.line(Y = Y_data)

## Line Plot

In [20]:
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data, X = X_data)

## Line Update

In [21]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y = Y_append, X =X_append, win=plt, update="append")

'window_39d440fe7ffc4e'

## Multiple Line on Single Windows

In [25]:
num = torch.Tensor(list(range(0, 10)))
num = num.view(-1 ,1)
num = torch.cat((num, num), dim = 1)

plt = vis.line(Y = torch.randn(10, 2), X = num)

## Line info

In [26]:
plt = vis.line(Y = Y_data, X = X_data, opts = dict(title = 'Test', showlegend = True))

In [27]:
plt = vis.line(Y=Y_data, X = X_data, opts= dict(title='Test', legend = ['no.1'], showlegend=True))

In [28]:
plt = vis.line(Y=torch.randn(10, 2), X = num, opts=dict(title = 'Test', legend = ['no.1', 'no.2'], showlegend = True))

## Make Function for update line

In [31]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num, Y = loss_value, win = loss_plot, update = 'append')

In [36]:
plt = vis.line(Y = torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

## Close the window

In [37]:
vis.close(env = "main")

''

Connection is already closed.
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
